In [25]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import torch
import torchvision
import torchvision.transforms as transforms

In [27]:
df = pd.read_csv('ADV.csv',index_col='customer_id')

In [29]:
#This version includes the debit ratio and the percent of cash transactions. The debit_percent feature is influential; removing it will change the number of clusters

In [31]:
df.head()

,industry_code,industry,industry_group,employee_count,sales,abm_num_debit,abm_avg_debit,abm_num_credit,abm_avg_credit,card_num_debit,...,eft_num_debit,eft_avg_debit,eft_num_credit,eft_avg_credit,emt_num_debit,emt_avg_debit,emt_num_credit,emt_avg_credit,cash_percent,debit_percent
customer_id,,,,,,,,,,,,,,,,,,,,,
SYNCID0000000000,7292,"Estate, Trust and Agency Funds",Investment Companies,0.0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,99.999554
SYNCID0000000001,NaN,NaN,NaN,NaN,66446.0,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,100.000000
SYNCID0000000002,7721,Computer Services,Management Consulting,2.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,52.164736
SYNCID0000000004,4565,Forest Products Trucking Industry,Trucking and Freight,1.0,328160.0,NaN,NaN,NaN,NaN,41.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,44.092805
SYNCID0000000005,7799,Other Business Services n.e.c.,Management Consulting,1.0,0.0,NaN,NaN,NaN,NaN,NaN,...,15.0,8363.572667,35.0,1137.053714,NaN,NaN,NaN,NaN,0.0,75.917236


In [33]:
df.fillna({'industry_code': 'Unknown'}, inplace=True)
df.fillna({'industry': 'Unknown'}, inplace=True) #could improve this bu putting other and null in different groups
df.fillna({'industry_group': 'Unknown'}, inplace=True) 

In [35]:
df['employee_count'] = df.groupby('industry_group')['employee_count'].transform(lambda x: x.fillna(x.mean()))
df['sales'] = df.groupby('industry_group')['sales'].transform(lambda x: x.fillna(x.mean()))

In [37]:
df.fillna(0.0, inplace=True)

In [39]:
df.head()

,industry_code,industry,industry_group,employee_count,sales,abm_num_debit,abm_avg_debit,abm_num_credit,abm_avg_credit,card_num_debit,...,eft_num_debit,eft_avg_debit,eft_num_credit,eft_avg_credit,emt_num_debit,emt_avg_debit,emt_num_credit,emt_avg_credit,cash_percent,debit_percent
customer_id,,,,,,,,,,,,,,,,,,,,,
SYNCID0000000000,7292,"Estate, Trust and Agency Funds",Investment Companies,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,99.999554
SYNCID0000000001,Unknown,Unknown,Unknown,2.16297,66446.000000,0.0,0.0,0.0,0.0,4.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,100.000000
SYNCID0000000002,7721,Computer Services,Management Consulting,2.00000,193864.737815,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,52.164736
SYNCID0000000004,4565,Forest Products Trucking Industry,Trucking and Freight,1.00000,328160.000000,0.0,0.0,0.0,0.0,41.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,44.092805
SYNCID0000000005,7799,Other Business Services n.e.c.,Management Consulting,1.00000,0.000000,0.0,0.0,0.0,0.0,0.0,...,15.0,8363.572667,35.0,1137.053714,0.0,0.0,0.0,0.0,0.0,75.917236


In [41]:
df_numeric = df.select_dtypes(include=['number'])

In [79]:
#Scale the features with minmax scaler
scaler = MinMaxScaler()
data_scaled = pd.DataFrame(scaler.fit_transform(df_numeric), columns=df_numeric.columns)
X_tensor = torch.tensor(data_scaled.values, dtype=torch.float32)


In [81]:
import torch.nn as nn
import torch.nn.functional as F

class Autoencoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [83]:
input_dim = df_numeric.shape[1]
hidden_dim = int(df_numeric.shape[1]/4)
output_dim = df_numeric.shape[1]
print(input_dim, hidden_dim, output_dim)
model = Autoencoder(input_dim, hidden_dim, output_dim)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train model
for epoch in range(100):
    for i in range(0, len(X_tensor), 120):  # Manually batch data
        x = X_tensor[i:i+120]
        # Forward pass
        reconstructed = model(x)
        # Calculate loss
        loss = criterion(reconstructed, x)
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # Print loss
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

28 7 28
Epoch 1, Loss: 0.010673296637833118
Epoch 2, Loss: 0.004530149977654219
Epoch 3, Loss: 0.003969188313931227
Epoch 4, Loss: 0.0027546538040041924
Epoch 5, Loss: 0.0014771046116948128
Epoch 6, Loss: 0.0014204547042027116
Epoch 7, Loss: 0.0014080397086218
Epoch 8, Loss: 0.0013998789945617318
Epoch 9, Loss: 0.00138140341732651
Epoch 10, Loss: 0.0012076105922460556
Epoch 11, Loss: 0.0011423338437452912
Epoch 12, Loss: 0.0011123635340481997
Epoch 13, Loss: 0.0010852465638890862
Epoch 14, Loss: 0.0010577515931800008
Epoch 15, Loss: 0.0010296913096681237
Epoch 16, Loss: 0.0010023185750469565
Epoch 17, Loss: 0.0009797661332413554
Epoch 18, Loss: 0.0009658495546318591
Epoch 19, Loss: 0.0009579525212757289
Epoch 20, Loss: 0.0009526805952191353
Epoch 21, Loss: 0.0009480940643697977
Epoch 22, Loss: 0.0009441302972845733
Epoch 23, Loss: 0.0009406894678249955
Epoch 24, Loss: 0.0009371276246383786
Epoch 25, Loss: 0.0009336176444776356
Epoch 26, Loss: 0.0009302666294388473
Epoch 27, Loss: 0.000

In [85]:
with torch.no_grad():
    reconstructed = model(X_tensor)
    reconstruction_error = torch.mean((X_tensor - reconstructed) ** 2, dim=1)  # MSE per sample

# Set a threshold for outliers (e.g., 95th percentile)
threshold = torch.quantile(reconstruction_error, 0.95)
outliers = reconstruction_error > threshold

print(f"Detected {outliers.sum().item()} outliers")

Detected 813 outliers


In [89]:
df["outlier"] = outliers.numpy().astype(int)

In [103]:
outlier_array = df[df["outlier"]==True].index.to_numpy()
outlier_array

array(['SYNCID0000000013', 'SYNCID0000000018', 'SYNCID0000000056',
       'SYNCID0000000057', 'SYNCID0000000060', 'SYNCID0000000068',
       'SYNCID0000000084', 'SYNCID0000000107', 'SYNCID0000000117',
       'SYNCID0000000150', 'SYNCID0000000212', 'SYNCID0000000213',
       'SYNCID0000000264', 'SYNCID0000000269', 'SYNCID0000000380',
       'SYNCID0000000381', 'SYNCID0000000418', 'SYNCID0000000431',
       'SYNCID0000000445', 'SYNCID0000000447', 'SYNCID0000000470',
       'SYNCID0000000486', 'SYNCID0000000491', 'SYNCID0000000517',
       'SYNCID0000000532', 'SYNCID0000000533', 'SYNCID0000000600',
       'SYNCID0000000619', 'SYNCID0000000627', 'SYNCID0000000648',
       'SYNCID0000000652', 'SYNCID0000000686', 'SYNCID0000000692',
       'SYNCID0000000702', 'SYNCID0000000703', 'SYNCID0000000717',
       'SYNCID0000000753', 'SYNCID0000000760', 'SYNCID0000000764',
       'SYNCID0000000776', 'SYNCID0000000829', 'SYNCID0000000831',
       'SYNCID0000000844', 'SYNCID0000000869', 'SYNCID00000008

In [113]:
np.savetxt("anomolies_AutoEncoder.csv", outlier_array, fmt='%s', delimiter=",")

In [117]:
test = np.loadtxt('anomolies_withDebitRatio10pct.csv',delimiter=',',dtype=str)
test

array(['SYNCID0000000060', 'SYNCID0000000068', 'SYNCID0000000107', ...,
       'SYNCID0000017139', 'SYNCID0000017174', 'SYNCID0000017175'],
      dtype='<U16')

In [121]:
intersection = np.intersect1d(outlier_array, test)  # Finds common elements
print(len(intersection))

460


In [123]:
print(len(outlier_array))

813
